This notebook follows the following tutorial about adding upscaling layers to a deep learning network so that I become more familiar with upsampling and downsampling: https://machinelearningmastery.com/upsampling-and-transpose-convolution-layers-for-generative-adversarial-networks/

## **Need for Upsampling in Generative Adversial Networks**

**Generative Adversarial Networks** are an architecture for neural networks for training a generative model. 

Architecture Composition (contained in a CNN):
- generator model -> generate new examples from the problem domain
- discrimator mode

### **Example Using the UpSampling2D Layer**
- The **simplest way** to **upsample** an input is to **double** each row and column


In [ ]:
# example of using the upsampling layer
from numpy import asarray
from keras.models import Sequential
from keras.layers import UpSampling2D

# define input data
# input image is 2 x 2 pixels
X = asarray([[1, 2],
 [3, 4]])

# show input data for context
print(X)

[[1 2]
 [3 4]]


In [ ]:
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

In [ ]:
# define model
model = Sequential()
model.add(UpSampling2D(input_shape=(2, 2, 1)))
# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 4, 4, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# make a prediction with the model
# upsampling a provided input image
yhat = model.predict(X)
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1. 1. 2. 2.]
 [1. 1. 2. 2.]
 [3. 3. 4. 4.]
 [3. 3. 4. 4.]]


In [ ]:
# example of using different scale factors for each dimension
model.add(UpSampling2D(size=(2, 3)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 4, 4, 1)           0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 12, 1)          0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 16, 36, 1)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 32, 108, 1)        0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 64, 324, 1)        0         
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 128, 972, 1)       0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 256, 2916, 1)      0

In [ ]:
# example of using bilinear interpolation when upsampling
model.add(UpSampling2D(interpolation='bilinear'))


### **Simple Generator Model With the UpSampling2D Layer**

- **UpSampling2D** layer is **simple** and **effective**, BUT does **not** perform any **learning**
- to be useful in GAN: each **"UpSampling2D"** need to be followed with a **"Conv2D layer**"
- the **Conv2D layer** will learn to interpret the doubled input and be trained to translate it


Example: produce a 10×10 image and take a 100 element vector as input

In [ ]:
# example of using upsampling in a simple generator model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Conv2D

# define model
model = Sequential()

# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))

# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

# double input from 128 5x5 to 1 10x10 feature map
model.add(UpSampling2D())

# fill in detail in the upsampled feature maps and output a single image
model.add(Conv2D(1, (3,3), padding='same'))

# summarize model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3200)              323200    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
up_sampling2d_18 (UpSampling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


1. Dense layer outputs 3,200  activations 
2. These activations are reshaped into 128 feature maps with the shape 5 x 5
3. The UpSampling 2D layer doubles the widths and heights whcih results in a feature map with 4X the area
4. Conv2D layer processes the feature maps and outputs a 10 x 10 image

## **Conv2DTranspose Layer**
- A layer that combines the UpSampling2D and Conv2D layers into one layer
- Referred to as deconvolution or deconvolutional layer 
- The transpose convolutional layer is like an inverse convolutional layer. 
    - Ex. A 2×2 stride would upsample the input instead of downsample
    



```
# With an output stride of (2,2):  the 1×1 convolution requires the insertion of additional rows and columns into the input image so that the reads of the operation can be performed

         1, 0, 2, 0
Input = (0, 0, 0, 0)
         3, 0, 4, 0
         0, 0, 0, 0
# The model can then read across this input using an output stride of (2,2) and will output a 4×4 image
          1, 0, 2, 0
Output = (0, 0, 0, 0)
          3, 0, 4, 0
          0, 0, 0, 0
```



### **Example Using the Conv2DTranspose Layer**

In [ ]:
# example of using the transpose convolutional layer
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2DTranspose

# define input data
X = asarray([[1, 2],
 [3, 4]])

# show input data for context
print(X)

[[1 2]
 [3 4]]


In [ ]:
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

In [ ]:
# define model
model = Sequential()

# takes 2x2 grayscale images as input and outputs the result
# the layer below will both upsamples and performs convolution thus
# the number of filters and the sze of the filters must be specified
model.add(Conv2DTranspose(1, (1,1), strides=(2,2), input_shape=(2, 2, 1)))

# summarize the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_2 (Conv2DTr (None, 4, 4, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define weights that they do nothing
# the single weight is set to value of 1.0 and bias value of 0.0
weights = [asarray([[[[1]]]]), asarray([0])]

# store the weights in the model
model.set_weights(weights)

# make a prediction with the model
yhat = model.predict(X)

# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))

# summarize output
print(yhat)

[[1. 0. 2. 0.]
 [0. 0. 0. 0.]
 [3. 0. 4. 0.]
 [0. 0. 0. 0.]]


In [ ]:
# example of using padding to ensure that the output is only doubled
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', input_shape=(2, 2, 1)))

## **Simple Generator Model With the Conv2DTranspose Layer**

In [2]:
# example of using transpose conv in a simple generator model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.layers import Conv2D

# define model
model = Sequential()

# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))

# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

# double input from 128 5x5 to 1 10x10 feature map
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))

# summarize model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_1 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________
